In [ ]:
import numpy as np
from matplotlib import colors, cm, rc
from matplotlib import pyplot as plt
import pickle
import signac as sg
from scipy.special import gamma
import copy as copy
from global_functions import adjustmaps
import h5py
from itertools import product, combinations

# Define/load things non-specific to a given set of results
metric = 'lambda_s'
Aeff = 7.29
t_final = 300
ncell_tot = 87_993
c = 1.42
with sg.H5Store('shared_data.h5').open(mode='r') as sd:
    b_vec = np.array(sd['b_vec'])
tau_vec = b_vec * gamma(1+1/c)
tauc_methods = ["flat"]
results_pre = 'gte_thresh'

# Update global plotting parameters
rc('axes', labelsize=21)  # Font size for x and y labels
rc('axes', titlesize=16)
rc('xtick', labelsize=19)  # Font size for x-axis tick labels
rc('ytick', labelsize=19)  # Font size for y-axis tick labels
rc('lines', markersize=15)
rc('lines', linewidth=5.5)
rc('legend', fontsize=19)
rc('font', family='sans-serif')
rc('font', serif=['Computer Modern Sans Serif'] + plt.rcParams['font.serif'])
rc('font', weight='light')
histlw = 5.5
cbar_lpad = 30
dpi = 40

# Function to read in things specific to given results as global variables
def set_globals(results_pre):
    if metric == 'lambda_s':
        globals()['metric_lab'] = r'$S$'
        globals()['rob_metric_lab'] = r'$S^*$'
        globals()['mean_metric_lab'] = r'$\bar{\lambda}(\tau)$'
    if metric == 'P_s':
        globals()['metric_lab'] = r'$S_{meta}$'
        globals()['rob_metric_lab'] = r'$\S_{meta}^*$'
        globals()['mean_metric_lab'] = r'$<P_s>$'
    if metric == 's':
        globals()['metric_lab'] = r'$s_{meta}$'
        globals()['rob_metric_lab'] = r'$\s_{meta}^*$'
        globals()['mean_metric_lab'] = r'$<s>$'
    globals()['fn_prefix'] = f"{results_pre}/data/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"
    globals()['fig_prefix'] = f"{results_pre}/figs/Aeff_{Aeff}/tfinal_{t_final}/metric_{metric}/"
    #globals()['fig_prefix'] = os.path.join('/','Volumes', 'Macintosh HD', 'Users', 'patrick',
    #                                       'Google Drive', 'My Drive', 'Research', 'Regan', 'Figs/')

# Read in maps and convert fdm to tau, used by multiple plots below
ul_coord = [1500, 2800]
lr_coord = [2723, 3905]
usecols = np.arange(ul_coord[0],lr_coord[0])
sdmfn = "../shared_maps/SDM_1995.asc"
sdm = np.loadtxt(sdmfn,skiprows=6+ul_coord[1],
                         max_rows=lr_coord[1], usecols=usecols)
fdmfn = '../shared_maps/FDE_current_allregions.asc'
fdm = np.loadtxt(fdmfn,skiprows=6+ul_coord[1],
                         max_rows=lr_coord[1], usecols=usecols)
sdm, fdm = adjustmaps([sdm, fdm])
delta_t = 30
b_raster = delta_t / np.power(-np.log(1-fdm), 1/c)
tau_raster = b_raster * gamma(1+1/c)
maps_filt = (sdm > 0) & (fdm > 0)
tau_flat = tau_raster[maps_filt]
mapindices = np.argwhere(maps_filt)
tau_argsort = np.argsort(tau_flat)
tau_sorted = tau_flat[tau_argsort]

# Define keys and labels for parameters
uncertain_params = ['mu_tau', 'sigm_tau', 'mu_tauc', 'sigm_tauc', 'mean_lam_diff']
param_labels = [r'$\mu_{\tau}$', r'$\sigma_{\tau}$', r'$\mu_{\hat{\tau}}$', 
                r'$\sigma_{\hat{\tau}}$', r'$<\lambda_m - \bar{\lambda}>$']

# Get all possible parameter pairs
param_pairs = [pair for pair in combinations(range(len(uncertain_params)), 2)]

# Read in data of S samples at optimal decisions, as well as some other things
set_globals(results_pre)

# Read in all splined interpolations of metric(tau)
with open(fn_prefix + "/metric_spl_all.pkl", "rb") as handle:
    metric_spl_all = pickle.load(handle)

In [ ]:
lam_baseline = metric_spl_all[0]
tau_samples = np.linspace(tau_vec[1], tau_vec.max(), 1_000)
tau_crit_i = np.abs(lam_baseline(tau_samples) - 0.975).argmin()
tau_crit = tau_samples[tau_crit_i]
tau_crit

Now plot the optimal value of tauhat_h under baseline conditions across R values. See if it matches tau_crit above.

In [ ]:
C_vec_baseline = np.load(fn_prefix + 'C_vec_baseline.npy')
x_all = np.load(fn_prefix + '/x_all_baseline.npy')
meta_metric_all = np.load(fn_prefix + '/meta_metric_all_baseline.npy')
meta_metric_all = meta_metric_all[:,0]
meta_metric_nochange = float(np.load(fn_prefix + 'meta_metric_nochange.npy'))

n_opt_vec = np.ones_like(C_vec_baseline) * np.nan
l_opt_vec = np.ones_like(C_vec_baseline) * np.nan
for C_i, C in enumerate(C_vec_baseline):
    zeroeps_filt = np.all(x_all[:, 3:] == 0, axis=1)
    _filt = zeroeps_filt & (x_all[:,0] == C)
    argmax = np.nanargmax(meta_metric_all[_filt])
    n_opt_vec[C_i] = x_all[_filt,:][argmax][1]
    l_opt_vec[C_i] = x_all[_filt,:][argmax][2]
tau_h = tau_sorted[(l_opt_vec + n_opt_vec).astype(int)]

In [ ]:
plt.axhline(tau_crit, c='k', ls='--', zorder=-1, label=r'critical baseline $\tau$')
plt.scatter(C_vec_baseline/ncell_tot, tau_h)
plt.ylim(0, 60)
plt.xlabel(r'$R~/~n_{tot}$')
plt.ylabel(r'optimal $\hat{\tau_h}$')
plt.legend()
plt.savefig(fig_prefix + '/baselineopttauh.png', bbox_inches='tight', dpi=50)

In [ ]:
tau_l = tau_sorted[l_opt_vec.astype(int)]
deltatau_vec = C_vec_baseline / n_opt_vec

plt.axhline(tau_crit, c='k', ls='--', zorder=-1, label=r'critical baseline $\tau$')
plt.scatter(C_vec_baseline/ncell_tot, tau_l, label=r'optimal $\hat{\tau}_l$')
plt.scatter(C_vec_baseline/ncell_tot, tau_l + deltatau_vec, label=r'optimal $\hat{\tau}_l+\Delta\tau$')
# plt.ylim(15, 35)
plt.ylim(0, 60)
plt.xlabel(r'$R~/~n_{tot}$')
# plt.ylabel(r'optimal $\hat{\tau_h}$')
plt.legend()

In [ ]:
tau_l + deltatau_vec

Get at 'intrinsic' lambda under baseline, i.e. baseline parameters, no fires

In [ ]:
project = sg.get_project()
jobs = project.find_jobs()
demographic_indices = []
for job in project:
    if job.sp.demographic_index == 0:
        with job.data as data:
            print(data['lambda_s/0'])